In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from sklearn.model_selection import train_test_split
from textblob.classifiers import NaiveBayesClassifier as NBC
from textblob import TextBlob
from nltk.sentiment.util import mark_negation
import re

C:\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
from multiprocessing import Pool
import warnings
warnings.simplefilter('ignore')

In [ ]:
data = pd.read_csv('train.csv')
SAdataset = pd.read_csv('Sentiment-Analysis-Dataset/Sentiment Analysis Dataset.csv', error_bad_lines=False, usecols = [1,3])
index0 = SAdataset.loc[SAdataset.Sentiment == 0]['Sentiment'] 
index1 = SAdataset.loc[SAdataset.Sentiment == 1]['Sentiment'] 
index0.ix[:] = 'neg'
index1.ix[:] = 'pos'
SAdataset.loc[index0.index, 'Sentiment'] = index0
SAdataset.loc[index1.index, 'Sentiment'] = index1

ind_neg = SAdataset[SAdataset.Sentiment == 'neg'].index[:5000]
ind_pos = SAdataset[SAdataset.Sentiment == 'pos'].index[:5000]

short_SAdataset = SAdataset.loc[np.hstack((ind_neg, ind_pos)), ['Sentiment', 'SentimentText']]

X = short_SAdataset['SentimentText']
y = short_SAdataset['Sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
train = [(X_train[index_id], y_train[index_id]) for index_id in y_train.index]
test = [(X_test[index_id], y_test[index_id]) for index_id in y_test.index]

cl = NBC(train)
cl.accuracy(test[:100])

data['sentiment'] = 0.0

def sentiment_analysis_tweet(classifier, text):
    return (cl.prob_classify(text).prob("neg"))

In [ ]:
number_of_processes = 2
with Pool(number_of_processes) as p:
    result = p.map(lambda x: sentiment_analysis_tweet(cl, x), data['comment_text'])

In [ ]:
data['sentiment'] = result
data.to_csv('train_tweets.csv', encoding='utf-8')

In [53]:
cl.show_informative_features() 

Most Informative Features
   contains(musicmonday) = True              pos : neg    =     48.8 : 1.0
           contains(sad) = True              neg : pos    =     39.0 : 1.0
         contains(hurts) = True              neg : pos    =     22.1 : 1.0
         contains(broke) = True              neg : pos    =     15.3 : 1.0
  contains(followfriday) = True              pos : neg    =     13.9 : 1.0
          contains(quot) = True              pos : neg    =     13.9 : 1.0
  contains(FollowFriday) = True              pos : neg    =     12.5 : 1.0
         contains(phone) = True              neg : pos    =     11.5 : 1.0
          contains(Love) = True              pos : neg    =     11.0 : 1.0
         contains(tried) = True              neg : pos    =     10.6 : 1.0
